# 特征构造与测试

本文档将叙述构造中高频特征并快速测试的一般流程

In [1]:
import pandas as pd
import sys
sys.path.append('C:\\Users\\boyu.deng\\Desktop\\d1\\high_frec\\factor_test')
from DataLoader.DataLoader import DataLoader
from DataProcess.DataProcessor_3 import DataProcessor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import sys

# 将你的本地路径添加在下面
# sys.path.append('C:/Users/Administrator/Desktop/Repositories/High-Frequency-Predictor')

from hfp.hfp import HFP

In [9]:
import hfp.hfp
importlib.reload(hfp.hfp)

from hfp.hfp import HFP

#### 初始化测试器

In [2]:
data_path = 'C:\\Users\\boyu.deng\\Desktop\\d1\\hfrec_data'  # 原始数据的路径
stock_num = 1  
hfp = HFP(data_path=data_path, stock_num=stock_num, stock_list= ['BTCUSDT'], start_date='2025-01-01', end_date='2025-08-28', interval='1m')

### 快速构造测试特征

#### 例子：
在以下例子中，我们测试了三个因子表达式的表现。通过调整shift字段可以查看对未来不同时间跨度的收益率的预测力。

In [6]:
from numpy import empty
from tqdm import tqdm

fml_list = ['tscorr{taker_buy_quote_volume,volume_ratio,360}','neg{tsfftreal{close,15,10}}','tsmean{close,15}']
factor_stats = {name:[] for name in ['key', 'formula', 'mean_corr','positive_corr_ratio','corr_IR']}
combined_signal_df = pd.DataFrame()

for fml in tqdm(fml_list[:], desc="测试公式进度"):
    try:
        stats, signal_df, corr = hfp.test_factor(fml,verbose=False, shift=0, start = 10, end = 1400)  # shift表明对下几个interval的收益率进行预测
    except Exception as e:
        print(e)
        continue
    #print(corr.shape)
    for key, value in stats.items():
        factor_stats['key'].append(key)
        factor_stats['formula'].append(fml)
        factor_stats['mean_corr'].append(value[0].mean_corr)
        factor_stats['positive_corr_ratio'].append(value[0].positive_corr_ratio)
        factor_stats['corr_IR'].append(value[0].corr_IR)
factor_df = pd.DataFrame(factor_stats)
factor_df

测试公式进度: 100%|██████████| 3/3 [00:04<00:00,  1.42s/it]


,key,formula,mean_corr,positive_corr_ratio,corr_IR
0,BTCUSDT,"tscorr{taker_buy_quote_volume,volume_ratio,360}",0.196381,0.750000,0.642298
1,BTCUSDT,"neg{tsfftreal{close,15,10}}",0.064728,0.795833,0.857566
2,BTCUSDT,"tsmean{close,15}",-0.067506,0.204167,-0.876929
